In [17]:
import pandas as pd
import numpy as np
from gensim.models import FastText
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import platform
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

In [2]:
data_path = '../../../data/뉴스'

train_token_path = os.path.join(data_path, 'token.csv')

train_token = pd.read_csv(train_token_path)

In [3]:
# 문장 전처리 함수
# 정규표현식으로 전처리 함
# default로 숫자, 한글, 영어 제외 모두 제거
# pandas의 apply 함수와 함께 사용해도 좋을 듯
# x : 전처리할 문장
# reg : 정규표현식
def reg_preprocessing(x, reg=r'[^\d가-힣a-zA-Z ]'):
    x = re.sub(reg, '', x)

    return x


# 문장을 토큰화 한 후 index로 변경
# 문장을 sequence로 변경
# text : 토큰화 할 text(train set)
# max_len : 훈련 시킬 문장의 최대 길이
# oov : 처음 보는 단어에 대한 token. fasttext는 필요 x
# num_words : 사용할 단어의 최소 빈도 수. model fit 할 때의 수랑 맞춰야 함
# pad_option : padding 및 trunc 때 앞에서 부터 할지 뒤에서 부터 할지
# 필요 module
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
def make_token_and_tokenizer(text, max_len=100, oov=None, num_words=None, pad_option='post'):
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov)
    tokenizer.fit_on_texts(text)
    token_index = tokenizer.texts_to_sequences(text)
    vocab_size = len(tokenizer.word_index) + 1
    token_index = pad_sequences(token_index, maxlen=max_len, padding=pad_option, truncating=pad_option)

    return token_index, vocab_size, tokenizer


# tf Embedding layer에 weight로 사용할 matrix를 만드는 함수
# word_dict : 단어와 인덱스 번호. tokenizer.word_index. 0은 padding
# model : embedding 모델(fasttext를 생각하고 만듦)
# vocab_size : make_token_and_tokenizer에서 얻어지는 vocab_size.
# embedding_size : 임베딩 차원. fasttext는 100이 default
# 필요 module
# 없음. 모델만 잘 넣어주면 됨
def make_embedding_matrix(word_dict, model, vocab_size, embedding_size=100):
    embedding_matrix = np.zeros((vocab_size, embedding_size))

    for word, index in word_dict.items():
        embedding_matrix[index] = model.wv[word]

    return embedding_matrix

In [4]:
# apply를 통한 전처리
train_token['title'] = train_token['title'].apply(reg_preprocessing)
train_token['title']

0                   인천  핀란드 항공기 결항 휴가 철 여행객 분통
1              실리콘밸리 넘어서겠다 구글 15조원 들여  전역 거점 화
2              이란 외무 긴장 완화 해결 책 미국 경제 전쟁 멈추는 것
3         NYT 클린턴 측근  기업 특수 관계 조명 공과 사 맞 물려 종합
4                    시진핑 트럼프 중미 무역 협상 조속 타결 희망
                         ...                  
45649             KB 금융 미국 IB 스티펠 제휴 선진국 시장 공략
45650        1 보 서울시 교육청 신종 코로나 확산 개학 연기 휴업 검토
45651          게시판 키움 증권 2020 키움 영웅 전 실전 투자 대회
45652                   답변 하는 배 기동 국립 중앙 박물관 장
45653    2020 한국 인터넷 기자 상 시상식 내달 1일 개최 특별상 김성후
Name: title, Length: 45654, dtype: object

In [5]:
# 리스트로 변환
tokenize_data = []
for sentence in train_token['title'].tolist():
    tokenize_data.append(sentence.split(' '))

In [7]:
# fasttext fit
model = FastText(tokenize_data, window=5, min_count=5, workers=4, sg=1)

In [8]:
# tokenizer 생성
token_index, vocab_size, tokenizer = make_token_and_tokenizer(train_token['title'].tolist(), num_words=5)

In [9]:
# embedding matrix 생성
embedding_matrix = make_embedding_matrix(tokenizer.word_index, model, vocab_size)
embedding_matrix[1]

array([-0.5281601 ,  0.21827047, -0.09583736, -0.05389892,  0.05651149,
       -0.32731995, -0.30143198,  0.31808397,  0.07005626,  0.06872333,
        0.27182803, -0.31650779,  0.03053061,  0.0317923 , -0.18763359,
       -0.10731325,  0.21604368, -0.35591209,  0.14379723, -0.07971345,
        0.31946507, -0.40950027, -0.12266451, -0.32632136, -0.12529513,
        0.22423929,  0.1129882 ,  0.11344469, -0.34932217, -0.38271603,
       -0.17162028, -0.1896355 ,  0.21483681, -0.12851153, -0.00591639,
       -0.08763758,  0.26277113, -0.3409645 , -0.02513706, -0.32403067,
       -0.00354818, -0.03306084, -0.33180583,  0.3900927 ,  0.10460509,
        0.18834165,  0.36209768, -0.05974977,  0.02914863,  0.00660153,
        0.15025064, -0.15514784, -0.30007142, -0.14558642,  0.03472638,
       -0.11497443,  0.47033048,  0.01644604, -0.2183286 ,  0.29826608,
        0.05926862, -0.02271334,  0.23134229,  0.2029067 ,  0.02538442,
        0.20853975,  0.73334974, -0.13216019,  0.00694963,  0.11

In [13]:
# 종합 index는 1
tokenizer.word_index['종합']

1

In [15]:
# embedding matrix와 동일
model.wv['종합']

array([-0.5281601 ,  0.21827047, -0.09583736, -0.05389892,  0.05651149,
       -0.32731995, -0.30143198,  0.31808397,  0.07005626,  0.06872333,
        0.27182803, -0.3165078 ,  0.03053061,  0.0317923 , -0.18763359,
       -0.10731325,  0.21604368, -0.3559121 ,  0.14379723, -0.07971345,
        0.31946507, -0.40950027, -0.12266451, -0.32632136, -0.12529513,
        0.22423929,  0.1129882 ,  0.11344469, -0.34932217, -0.38271603,
       -0.17162028, -0.1896355 ,  0.2148368 , -0.12851153, -0.00591639,
       -0.08763758,  0.26277113, -0.3409645 , -0.02513706, -0.32403067,
       -0.00354818, -0.03306084, -0.33180583,  0.3900927 ,  0.10460509,
        0.18834165,  0.36209768, -0.05974977,  0.02914863,  0.00660153,
        0.15025064, -0.15514784, -0.30007142, -0.14558642,  0.03472638,
       -0.11497443,  0.47033048,  0.01644604, -0.2183286 ,  0.29826608,
        0.05926862, -0.02271334,  0.23134229,  0.2029067 ,  0.02538442,
        0.20853975,  0.73334974, -0.13216019,  0.00694963,  0.11

In [18]:
embed_layer = keras.layers.Embedding(vocab_size, 100, weights=[embedding_matrix])

In [19]:
# embedding matrix, model의 종합 벡터와 동일
embed_layer(1)

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([-0.5281601 ,  0.21827047, -0.09583736, -0.05389892,  0.05651149,
       -0.32731995, -0.30143198,  0.31808397,  0.07005626,  0.06872333,
        0.27182803, -0.3165078 ,  0.03053061,  0.0317923 , -0.18763359,
       -0.10731325,  0.21604368, -0.3559121 ,  0.14379723, -0.07971345,
        0.31946507, -0.40950027, -0.12266451, -0.32632136, -0.12529513,
        0.22423929,  0.1129882 ,  0.11344469, -0.34932217, -0.38271603,
       -0.17162028, -0.1896355 ,  0.2148368 , -0.12851153, -0.00591639,
       -0.08763758,  0.26277113, -0.3409645 , -0.02513706, -0.32403067,
       -0.00354818, -0.03306084, -0.33180583,  0.3900927 ,  0.10460509,
        0.18834165,  0.36209768, -0.05974977,  0.02914863,  0.00660153,
        0.15025064, -0.15514784, -0.30007142, -0.14558642,  0.03472638,
       -0.11497443,  0.47033048,  0.01644604, -0.2183286 ,  0.29826608,
        0.05926862, -0.02271334,  0.23134229,  0.2029067 ,  0.02538442,
        0.208539